In [ ]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
from topoPy import *

df = nc.Dataset('../data/icon_grid_0010_R02B04_G_linked.nc')

clat = df.variables['clat'][:]
clon = df.variables['clon'][:]
clat_verts = df.variables['clat_vertices'][:]
clon_verts = df.variables['clon_vertices'][:]
links = df.variables['links'][:]

# clat = clat*(180/np.pi)
# clon = clon*(180/np.pi)
# clat_verts = clat_verts*(180/np.pi)
# clon_verts = clon_verts*(180/np.pi)

datfile = '../data/GMTED2010_topoGlobal_SGS_30ArcSec.nc'
var = {'name':'topo','units':'m'}

np.random.seed(555)
# icon_cell_indexes = np.sort([440, 19442, 5595, 5026, 4793, 4631])
# icon_cell_indexes = np.random.randint(0,np.size(clat)-1,36)
icon_cell_indexes = np.array([  343,  1021,  1367,  2045,  2391,  3069,  3415,  4093,  4439,
        5117,  5588,  5603,  5985,  6012,  6612,  6627,  7009,  7036,
        7636,  7651,  8033,  8060,  8660,  8675,  9057,  9084,  9684,
        9699, 10081, 10108]) # cells that are not being found on the grid...

# icon_cell_indexes = [3027,3028,3029]
# Mount Ebrus; Firehorn; Taunus; Pirin; Langtang; ???
print(icon_cell_indexes)
print(icon_cell_indexes.shape)

In [ ]:
comp_clat = clat[icon_cell_indexes]
comp_clon = clon[icon_cell_indexes]
comp_clat_verts = clat_verts[icon_cell_indexes]
comp_clon_verts = clon_verts[icon_cell_indexes]

ncfile = Dataset('../data/icon_compact.nc',mode='w') 
print(ncfile)

cell = ncfile.createDimension('cell', np.size(comp_clat))     # latitude axis
nv = ncfile.createDimension('nv', 3)    # longitude axis
for dim in ncfile.dimensions.items():
    print(dim)

ncfile.title='Compact ICON grid for testing and debugging purposes'
print(ncfile.title)

clat = ncfile.createVariable('clat', np.float32, ('cell',))
clat.units = 'radian'
clat.long_name = 'center latitude'

clon = ncfile.createVariable('clon', np.float32, ('cell',))
clon.units = 'radian'
clon.long_name = 'center longitude'

clat_verts = ncfile.createVariable('clat_vertices', np.float32, ('cell','nv',))
clat_verts.units = 'radian'

clon_verts = ncfile.createVariable('clon_vertices', np.float32, ('cell','nv',))
clon_verts.units = 'radian'

clat[:] = comp_clat
clon[:] = comp_clon
clat_verts[:,:] = comp_clat_verts
clon_verts[:,:] = comp_clon_verts

print(clon_verts[:,:])

ncfile.close(); print('Dataset is closed!')

In [ ]:
links_tmp = links[icon_cell_indexes].flatten()
links_tmp = links_tmp[np.where(links_tmp > 0)]
links_tmp = np.sort(list(set(links_tmp)))
links_tmp -= 1

print(links_tmp)

lon, lat, z = readnc(datfile, var)
nrecords = np.shape(z)[0]; nlon = np.shape(lon)[1]; nlat = np.shape(lat)[1]

sz_tmp = np.size(links_tmp)
# print(nlat,nlon, np.size(links_tmp))

compactified_topo = np.zeros((sz_tmp,nlat, nlon))
print(compactified_topo.shape)

compactified_lat = np.zeros((sz_tmp,nlat))
compactified_lon = np.zeros((sz_tmp,nlon))
    
for i,lnk in enumerate(links_tmp):
    print("i, lnk = ", (i, lnk))
    compactified_lat[i] = lat[lnk]
    compactified_lon[i] = lon[lnk]
    compactified_topo[i] = z[lnk]
    
del lat, lon, z

ncfile = Dataset('../data/topo_compact.nc',mode='w',format='NETCDF4_CLASSIC') 
print(ncfile)

nfiles = ncfile.createDimension('nfiles', sz_tmp)
lat = ncfile.createDimension('lat', nlat)
lon = ncfile.createDimension('lon', nlon)
for dim in ncfile.dimensions.items():
    print(dim)

ncfile.title='Compact GMTED2010 USGS Topography grid for testing and debugging purposes'
print(ncfile.title)

lat = ncfile.createVariable('lat', np.float32, ('nfiles','lat'))
lat.units = 'degrees'

lon = ncfile.createVariable('lon', np.float32, ('nfiles','lon'))
lon.units = 'degrees'

topo = ncfile.createVariable('topo', np.float32, ('nfiles','lat','lon'))
topo.units = 'meters'

lat[:,:] = compactified_lat
lon[:,:] = compactified_lon
topo[:,:,:] = compactified_topo

ncfile.close(); print('Dataset is closed!')

In [ ]:
lon, lat, z = readnc(datfile, var)
nrecords = np.shape(z)[0]; nlon = np.shape(lon)[1]; nlat = np.shape(lat)[1]

In [ ]:
print(lon[1][:])
print(lon[2][:])
print(lon[3][:])
print(lon[4][:])

In [ ]:
print(lat[1][:])
print(lat[2][:])
print(lat[3][:])
print(lat[4][:])